# Closed Loop Kinematics

We want to simulate a closed loop system - just because . As a starting point, we will tackle the **STEWART PLATFORM**.
While this seems like an overkill, applying the closure conditions in pybullet is quite easy.

First, we import the needed modules.

In [1]:
import pybullet as pb
import pybullet_data as pbd

And start a server. Additionally, we supply the open loop model of the stewart platform.

In [2]:
client = pb.connect(pb.GUI)

robot_path = '../../models/urdf/Stewart.urdf'

We will start by simulating the open loop kinematics under the influence of gravity.

In [3]:
pb.resetSimulation()
pb.removeAllUserDebugItems()

pb.setGravity(0, 0, -9.81, client)

robot = pb.loadURDF(robot_path, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE|pb.URDF_USE_SELF_COLLISION_INCLUDE_PARENT)

pb.setRealTimeSimulation(1)

In [4]:
pb.resetSimulation()
pb.removeAllUserDebugItems()

robot = pb.loadURDF(robot_path, useFixedBase=True)

pb.setPhysicsEngineParameter(
    fixedTimeStep = 1e-4,
    numSolverIterations = 200,
    constraintSolverType = pb.CONSTRAINT_SOLVER_LCP_DANTZIG,
    globalCFM = 0.000001    
)
pb.setRealTimeSimulation(1)

In [5]:
# Gather all joints and links
joint_dict = {}
for i in range(pb.getNumJoints(robot, client)):
    joint_info = pb.getJointInfo(robot, i)
    name = (joint_info[1]).decode('UTF-8')
    if not name in joint_dict.keys():
        joint_dict.update(
        {name : i}
        )

# Make some new constraints
parents = ['top_11', 'top_12', 'top_13']
children = {'top_11' : ['ITF_31'], 'top_12' : ['ITF_22', 'ITF_12'], 'top_13' : ['ITF_23', 'ITF_33']}
constraint_dict = {}
for parent in parents:
    parent_id = joint_dict[parent]
    for child in children[parent]:
        constraint_name = parent + '_2_' + child 
        child_id = joint_dict[child]
        # Create a p2p connection
        constraint_id = pb.createConstraint(
            parentBodyUniqueId = robot,
            parentLinkIndex = parent_id,
            childBodyUniqueId = robot,
            childLinkIndex = child_id, 
            jointType = pb.JOINT_POINT2POINT,
            jointAxis = (0,0,0),
            parentFramePosition = (0,0,0),
            childFramePosition = (0,0,0),           
        )
        # Store the constraint information
        constraint_dict.update(
            {constraint_name : constraint_id}
        )
        # Change the constraint erp
        pb.changeConstraint(
            constraint_id,
            erp = 1e-4,
        )

In [6]:
# Create Actuation
control = ['P_11',  'P_33', 'P_55', 'P_22', 'P_44', 'P_55']
debug_dict = {}
for actor in control:
    actor_id = joint_dict[actor]
    joint_info = pb.getJointInfo(robot, actor_id)
    min_val = joint_info[8]
    max_val = joint_info[9]
    current_val = pb.getJointState(robot, actor_id, client)[0]
    debug_id = pb.addUserDebugParameter(actor, min_val, max_val, current_val, client)
    debug_dict.update(
        {debug_id : actor_id}
    )    

In [ ]:
# Enable Position control
while True:
    for debug, actor  in debug_dict.items():
        current_val = pb.readUserDebugParameter(debug, client)
        pb.setJointMotorControl2(
            robot,
            actor,
            controlMode = pb.POSITION_CONTROL,
            targetPosition = current_val,
        )